In [ ]:
import os
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt

from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import SGD, Adam
from keras.models import load_model, model_from_json

In [ ]:
! rm image_orientation -r
! git clone https://github.com/Ducvoccer/image_orientation.git

import image_orientation.utils
from image_orientation.utils import RotNetDataGenerator
from image_orientation.checkpointers import ModelCheckpoint

from importlib import reload
reload(image_orientation.utils)
reload(image_orientation.checkpointers)
from image_orientation.utils import RotNetDataGenerator
from image_orientation.utils import angle_error
from image_orientation.utils import angle_difference
from image_orientation.utils import generate_rotated_image
from image_orientation.checkpointers import ModelCheckpoint


In [ ]:
! mkdir models

In [ ]:
import zipfile
z = zipfile.ZipFile('/content/drive/My Drive/share_cv/Machine_learning/test_dataset/test_dataset.zip')
z.extractall('./')

In [ ]:
! cp /content/drive/My\ Drive/share_cv/Machine_learning/rotnet_COCO_resnet50.hdf5 ./models
! cp /content/drive/My\ Drive/share_cv/Machine_learning/train_1_backup/weight_ver3_phase1_16_0.06.hdf5 ./models/weight_ver3_phase1_16_0.06.hdf5
! cp /content/drive/My\ Drive/share_cv/Machine_learning/train_2_backup/weight_ver3_phase2_18_2.09_3.29.hdf5 ./models/weight_ver3_phase2_18_2.09_3.29.hdf5


In [ ]:
! cp /content/drive/My\ Drive/share_cv/Machine_learning/architecture_ver3/ ./models -r

In [ ]:
model_ver_1 = load_model('models/rotnet_COCO_resnet50.hdf5', custom_objects={'angle_error': angle_error})
# model_ver_2 = load_model()

with open('models/architecture_ver3/model_ver_3_phase_1.def') as f:
    model_ver_3_phase_1 = model_from_json(f.read())

with open('models/architecture_ver3/model_ver_3_phase_2.def') as f:
    model_ver_3_phase_2 = model_from_json(f.read())

model_ver_3_phase_1.load_weights('models/weight_ver3_phase1_16_0.06.hdf5')
model_ver_3_phase_2.load_weights('models/weight_ver3_phase2_18_2.09_3.29.hdf5')

In [ ]:
easy_image_paths  = ['test_dataset/easy/' + i for i in list(os.listdir(os.path.join('test_dataset', 'easy')))]
hard_image_paths  = ['test_dataset/hard/' + i for i in list(os.listdir(os.path.join('test_dataset', 'hard')))]

# Evaluate for model version 1

In [ ]:
test_data_easy=RotNetDataGenerator(
        easy_image_paths,
        input_shape=(224, 224, 3),
        batch_size=64,
        preprocess_func=preprocess_input,
        crop_center=True,
        crop_largest_rect=True,
        task_pharse = 3
)

scores_easy = model_ver_1.evaluate_generator(test_data,len(test_data))

test_data_hard=RotNetDataGenerator(
        hard_image_paths,
        input_shape=(224, 224, 3),
        batch_size=64,
        preprocess_func=preprocess_input,
        crop_center=True,
        crop_largest_rect=True,
        task_pharse = 3
)

scores_hard = model_ver_1.evaluate_generator(test_data,len(test_data))

print('Angle error: ', scores_easy[1])
print('Angle error: ', scores_hard[1])